# StateLog × TradeLog 解析テンプレート
- `analysis/env_data` と `analysis/bt_results` に配置した CSV を読み込み、環境指標とBT結果を突き合わせるためのノートです。
- `merged_trades` DataFrame を作成しておけば、帯域別・時間別の集計を容易に反復できます。

In [1]:
import pandas as pd
from pathlib import Path
from functools import lru_cache
from IPython.display import display, Markdown

def find_workspace_root(marker='analysis'):
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        if (candidate / marker).is_dir():
            return candidate
    raise FileNotFoundError("'{}' directory not found from {}".format(marker, current))

WORKSPACE_ROOT = find_workspace_root()
DATA_ROOT = WORKSPACE_ROOT / 'analysis'
ENV_DIR = DATA_ROOT / 'env_data'
BT_DIR = DATA_ROOT / 'bt_results'

def list_csv(directory, pattern='*.csv'):
    return sorted(directory.glob(pattern))

print('Workspace root:', WORKSPACE_ROOT)
print('Env dir:', ENV_DIR)
print('BT dir:', BT_DIR)
print('Detected env csv:', len(list_csv(ENV_DIR)))
print('Detected bt csv:', len(list_csv(BT_DIR, 'TradeLog_*.csv')))

def resolve_bt_path(bt_name=None, pattern='TradeLog_*.csv'):
    if bt_name:
        candidate = BT_DIR / bt_name
        if candidate.exists():
            return candidate
        raise FileNotFoundError(f"{candidate} が存在しません")
    files = list_csv(BT_DIR, pattern)
    if not files:
        raise FileNotFoundError(f"BT_DIR に {pattern} が見つかりません")
    return files[-1]

def load_bt_dataframe(bt_name=None):
    bt_path = resolve_bt_path(bt_name)
    df = pd.read_csv(bt_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('.', '-', regex=False))
    return df, bt_path

@lru_cache(maxsize=1)
def build_env_index():
    env_map = {}
    for csv_path in list_csv(ENV_DIR, '*.csv'):
        digits = ''.join(filter(str.isdigit, csv_path.stem))
        if len(digits) < 8:
            continue
        key = digits[-8:]
        env_map.setdefault(key, []).append(csv_path)
    return {k: tuple(v) for k, v in env_map.items()}

def load_env_frames(date_keys, sep=';'):
    env_map = build_env_index()
    frames = []
    missing = []
    for key in date_keys:
        paths = env_map.get(key)
        if not paths:
            missing.append(key)
            continue
        day_frames = [pd.read_csv(path, sep=sep) for path in paths]
        env_part = pd.concat(day_frames, ignore_index=True)
        env_part['bar_time'] = pd.to_datetime(env_part['bar_time'].str.replace('.', '-', regex=False))
        frames.append(env_part)
    return frames, missing

def concat_env_frames(frames):
    if not frames:
        raise ValueError('frames is empty')
    return pd.concat(frames, ignore_index=True).sort_values('bar_time')

def merge_entries_with_env(bt_df, env_df, tolerance='5min'):
    entries = bt_df[bt_df['event'] == 'ENTRY'].copy().sort_values('timestamp')
    env_sorted = env_df.sort_values('bar_time')
    merged = pd.merge_asof(entries, env_sorted, left_on='timestamp', right_on='bar_time',
                           direction='nearest', tolerance=pd.Timedelta(tolerance))
    return merged

def build_entry_exit(bt_df, merged):
    required = {'ticket', 'strategy'}
    missing = required - set(bt_df.columns)
    if missing:
        raise KeyError(f"BT CSVに必要な列がありません: {missing}")
    exit_columns = ['ticket', 'net', 'pips', 'timestamp']
    if 'exit_reason' in bt_df.columns:
        exit_columns.append('exit_reason')
    exits = bt_df[bt_df['event'] == 'EXIT'][exit_columns].copy()
    exits = exits.rename(columns={'timestamp': 'exit_timestamp', 'net': 'exit_net', 'pips': 'exit_pips'})

    merged_entries = merged.copy()
    redundant = [col for col in ('exit_reason', 'exit_timestamp', 'exit_net', 'exit_pips') if col in merged_entries.columns]
    if redundant:
        merged_entries = merged_entries.drop(columns=redundant)

    entry_exit = merged_entries.merge(exits, on='ticket', how='left')

    for col in ['atr_entry', 'adx_entry', 'donchian_width']:
        if col not in entry_exit.columns:
            entry_exit[col] = pd.NA

    width_candidates = [
        'donchian_width', 'donchian_width_x', 'donchian_width_entry', 'donchian_width_bt',
        'donchian_width_y', 'donchian_width_env', 'donchian_width_exit'
    ]
    width_series = None
    for col in width_candidates:
        if col in entry_exit.columns:
            data = entry_exit[col]
            width_series = data.copy() if width_series is None else width_series.fillna(data)
    if width_series is None:
        width_series = pd.Series(pd.NA, index=entry_exit.index)
    entry_exit['donchian_width'] = pd.to_numeric(width_series, errors='coerce')

    numeric_cols = ['atr_entry', 'adx_entry', 'donchian_width', 'exit_net', 'exit_pips']
    for col in numeric_cols:
        if col in entry_exit.columns:
            entry_exit[col] = pd.to_numeric(entry_exit[col], errors='coerce')

    if 'exit_reason' in entry_exit.columns:
        entry_exit['exit_reason'] = entry_exit['exit_reason'].fillna('UNKNOWN')
    else:
        entry_exit['exit_reason'] = 'UNKNOWN'

    return entry_exit

def ensure_entry_exit():
    notebook_globals = globals()
    entry_exit = notebook_globals.get('entry_exit')
    if entry_exit is not None:
        return entry_exit
    bt_df = notebook_globals.get('bt_df')
    merged = notebook_globals.get('merged')
    if bt_df is None or merged is None:
        raise NameError('entry_exit を生成するには bt_df と merged が必要です。')
    entry_exit = build_entry_exit(bt_df, merged)
    notebook_globals['entry_exit'] = entry_exit
    return entry_exit

ATR_BUCKET_BINS = [0, 0.10, 0.14, 0.18, 1]
ATR_BUCKET_LABELS = ['0.00-0.10','0.10-0.14','0.14-0.18','0.18+']
ADX_BUCKET_BINS = [0, 25, 30, 100]
ADX_BUCKET_LABELS = ['<=25','25-30','30+']
DONCHIAN_BUCKET_BINS = [0, 0.25, 1]
DONCHIAN_BUCKET_LABELS = ['low','high']

CURRENT_NOTEBOOK_PATH = WORKSPACE_ROOT / Path('analysis/notebooks/state_bt_analysis.ipynb')
OUTPUT_NOTEBOOK_DIR = CURRENT_NOTEBOOK_PATH.parent

pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 120)


Workspace root: /home/anyo_/workspace/YoYoEA_Multi_Entry
Env dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data
BT dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/bt_results
Detected env csv: 195
Detected bt csv: 2


In [2]:

# ==== 解析設定 ====
BT_FILE_NAME = None  # 解析対象のBTログ。最新ファイルを使う場合は None のまま
ENV_SEPARATOR = ';'  # StateLog CSV の区切り文字
MERGE_TOLERANCE = '5min'  # ENTRY時刻と環境データを突き合わせる許容時間
SAMPLE_PREVIEW_ROWS = 5  # ロード直後に表示するサンプル行数

pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 120)


In [3]:

# ==== BT/環境データの読み込み ====
bt_df, bt_path = load_bt_dataframe(BT_FILE_NAME)
print(f'Using BT file: {bt_path.name} ({len(bt_df)} rows)')

date_keys = sorted(bt_df['timestamp'].dt.strftime('%Y%m%d').unique())
env_frames, missing_dates = load_env_frames(tuple(date_keys), sep=ENV_SEPARATOR)
if missing_dates:
    print('[WARN] 環境ファイル未検出日:', ', '.join(missing_dates))

if not env_frames:
    raise FileNotFoundError('BT対象日に対応する環境ファイルが見つかりません')

env_df = concat_env_frames(env_frames)
print('Loaded env rows:', len(env_df), 'from', len(env_frames), '日分')

merged = merge_entries_with_env(bt_df, env_df, tolerance=MERGE_TOLERANCE)
print('merged rows', len(merged))
preview_cols = [col for col in ['timestamp_x', 'event', 'strategy', 'atr_entry', 'adx_entry', 'exit_reason'] if col in merged.columns]
if preview_cols:
    sample = merged[preview_cols].head(SAMPLE_PREVIEW_ROWS).fillna('-')
    print('--- merged sample ---')
    print(sample.to_string(index=False))
else:
    print('preview columns not available')


Using BT file: TradeLog_USDJPY_M15_ConfigOff_BE2_TrailOff_TP100_20250101_20250831.csv (2468 rows)
Loaded env rows: 18353 from 172 日分
merged rows 1236
--- merged sample ---
        timestamp_x event strategy  atr_entry adx_entry exit_reason
2025-01-01 00:00:00 ENTRY      CCI   0.049786         -           -
2025-01-02 01:53:17 ENTRY      CCI   0.081286         -           -
2025-01-02 01:53:17 ENTRY     MACD   0.081286         -           -
2025-01-02 02:12:40 ENTRY      RSI   0.103286         -           -
2025-01-02 04:15:00 ENTRY     MACD   0.149857         -           -


In [4]:

# ==== ENTRY環境とEXIT結果を結合 ====
entry_exit = build_entry_exit(bt_df, merged)
print('combined rows', len(entry_exit))
preview_cols = [col for col in ['timestamp_x', 'strategy', 'atr_entry', 'adx_entry', 'exit_reason', 'exit_net'] if col in entry_exit.columns]
if preview_cols:
    sample = entry_exit[preview_cols].head(SAMPLE_PREVIEW_ROWS).fillna('-')
    print('--- entry/exit sample ---')
    print(sample.to_string(index=False))


combined rows 1236
--- entry/exit sample ---
        timestamp_x strategy  atr_entry adx_entry    exit_reason  exit_net
2025-01-01 00:00:00      CCI   0.049786         -      STOP_LOSS    -31.73
2025-01-02 01:53:17      CCI   0.081286         - STOP_BREAKEVEN      1.90
2025-01-02 01:53:17     MACD   0.081286         - STOP_BREAKEVEN      1.90
2025-01-02 02:12:40      RSI   0.103286         - STOP_BREAKEVEN      1.90
2025-01-02 04:15:00     MACD   0.149857         - STOP_BREAKEVEN      1.90


/tmp/ipykernel_4901/2821409405.py:112: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  width_series = data.copy() if width_series is None else width_series.fillna(data)


In [5]:
# ==== セッション×指標サマリ ====
entry_exit = ensure_entry_exit()
entry_exit['exit_net'] = entry_exit['exit_net'].astype(float)

if 'session' not in entry_exit.columns:
    entry_exit['session'] = 'UNKNOWN'
else:
    entry_exit['session'] = entry_exit['session'].fillna('UNKNOWN')

if 'strategy' not in entry_exit.columns:
    raise KeyError('strategy 列が存在しません')

summary_cols = ['session', 'strategy']


def summarize(group):
    total = len(group)
    wins = group['exit_net'] > 0
    losses = group['exit_net'] < 0
    gross_profit = group.loc[wins, 'exit_net'].sum()
    gross_loss = group.loc[losses, 'exit_net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'exit_net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'exit_net'].mean() if losses.any() else 0.0
    expectancy = group['exit_net'].mean()
    return pd.Series({
        'Trades': total,
        'Wins': int(wins.sum()),
        'Losses': int(losses.sum()),
        'Win Rate (%)': win_rate * 100,
        'PF': profit_factor,
        'Avg Win': avg_win,
        'Avg Loss': avg_loss,
        'Expectancy': expectancy,
    })

session_strategy = (
    entry_exit.groupby(summary_cols, group_keys=False, observed=False)
    .apply(summarize, include_groups=False)
    .reset_index()
    .sort_values(['session', 'strategy'])
)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}

session_strategy.style.format(format_dict).set_table_attributes('style="table-layout: fixed; width: 100%;"')


,session,strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
0,ASIA,CCI,86.000000,55.000000,31.000000,63.95,0.97,18.55,-33.96,-0.38
1,ASIA,MACD,88.000000,51.000000,37.000000,57.95,0.93,22.73,-33.86,-1.06
2,ASIA,MA_CROSS,19.000000,12.000000,7.000000,63.16,0.67,13.02,-33.42,-4.09
3,ASIA,RSI,37.000000,26.000000,11.000000,70.27,1.56,22.52,-34.17,5.67
4,ASIA,STOCH,72.000000,42.000000,30.000000,58.33,1.00,24.08,-33.86,-0.06
5,EUROPE,CCI,100.000000,65.000000,35.000000,65.00,1.22,22.30,-33.97,2.60
6,EUROPE,MACD,93.000000,61.000000,32.000000,65.59,1.21,21.49,-33.89,2.43
7,EUROPE,MA_CROSS,25.000000,17.000000,8.000000,68.00,0.84,13.49,-34.16,-1.76
8,EUROPE,RSI,27.000000,16.000000,11.000000,59.26,0.25,5.91,-33.74,-10.24
9,EUROPE,STOCH,57.000000,29.000000,28.000000,50.88,0.41,13.19,-33.60,-9.80


In [6]:
# ==== ATR×ADX帯域の損益集計 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','strategy']).copy()
adx_candidates = []
if 'adx_entry' in attr.columns:
    adx_candidates.append(attr['adx_entry'])
if 'adx14' in attr.columns:
    adx_candidates.append(attr['adx14'])
if not adx_candidates:
    raise ValueError('ADX列が見つかりません (adx_entry/ adx14)')
adx_value = None
for series in adx_candidates:
    adx_value = series if adx_value is None else adx_value.combine_first(series)
attr['adx_value'] = adx_value
attr = attr.dropna(subset=['adx_value'])
if attr.empty:
    raise ValueError('ATR/ADX/strategy が不足しています')
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
attr['adx_bucket'] = pd.cut(attr['adx_value'].astype(float), bins=ADX_BUCKET_BINS, labels=ADX_BUCKET_LABELS, right=False)
metrics = attr.pivot_table(index=['atr_bucket','adx_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['count', :]] = metrics.loc[:, idx['count', :]].astype(int)
if 'sum' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['sum', :]] = metrics.loc[:, idx['sum', :]].round(2)
if 'mean' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['mean', :]] = metrics.loc[:, idx['mean', :]].round(2)
metrics.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [7]:
# ==== ATR帯×期間ばらつきサマリ ====
entry_exit = ensure_entry_exit()
exits = entry_exit.dropna(subset=['atr_entry','exit_net']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')

# 期間キー（優先: exit_timestamp -> timestamp_x -> timestamp）
ts_col = None
for cand in ('exit_timestamp', 'timestamp_x', 'timestamp'):
    if cand in exits.columns:
        ts_col = cand
        break
if ts_col is None:
    raise KeyError('時刻列が見つかりません (exit_timestamp/timestamp_x/timestamp)')

exits[ts_col] = pd.to_datetime(exits[ts_col])
exits['period'] = exits[ts_col].dt.to_period('M').dt.to_timestamp()
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)

def summarize_period(g):
    total = len(g)
    wins = g['exit_net'] > 0
    losses = g['exit_net'] < 0
    gross_profit = g.loc[wins, 'exit_net'].sum()
    gross_loss = g.loc[losses, 'exit_net'].sum()
    pf = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    expectancy = g['exit_net'].mean()
    return pd.Series({
        'trades': total,
        'win_rate_pct': win_rate * 100,
        'pf': pf,
        'expectancy': expectancy,
        'sum_net': g['exit_net'].sum(),
    })

period_stats = (
    exits.groupby(['atr_bucket','period'], observed=False)
    .apply(summarize_period)
    .reset_index()
    .sort_values(['atr_bucket','period'])
)

# 期間方向の分位点でばらつきを確認
band_quantiles = period_stats.groupby('atr_bucket', observed=False).agg({
    'trades': ['mean', 'median'],
    'win_rate_pct': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'pf': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'expectancy': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
})
band_quantiles.columns = [
    'trades_mean', 'trades_median',
    'win_rate_median', 'win_rate_p25', 'win_rate_p75',
    'pf_median', 'pf_p25', 'pf_p75',
    'expectancy_median', 'expectancy_p25', 'expectancy_p75',
]
band_quantiles = band_quantiles.round(2)
print('--- period_stats sample ---')
print(period_stats.head())
band_quantiles


--- period_stats sample ---
  atr_bucket     period  trades  win_rate_pct        pf  expectancy  sum_net
0  0.00-0.10 2025-01-01    28.0     75.000000  0.458865   -4.298929  -120.37
1  0.00-0.10 2025-02-01    25.0     84.000000  3.677196   14.238400   355.96
2  0.00-0.10 2025-03-01    16.0     62.500000  0.424975   -7.209375  -115.35
3  0.00-0.10 2025-04-01    12.0     66.666667  0.122762   -9.920833  -119.05
4  0.00-0.10 2025-05-01    21.0     52.380952  1.033319    0.550952    11.57


/tmp/ipykernel_4901/254438320.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_period)


,trades_mean,trades_median,win_rate_median,win_rate_p25,win_rate_p75,pf_median,pf_p25,pf_p75,expectancy_median,expectancy_p25,expectancy_p75
atr_bucket,,,,,,,,,,,
0.00-0.10,28.62,26.0,68.70,62.85,73.41,0.86,0.45,1.66,-1.76,-5.03,6.18
0.10-0.14,53.75,54.5,64.83,62.59,69.98,0.95,0.80,1.27,-0.50,-2.59,2.45
0.14-0.18,36.12,37.0,57.57,54.17,58.52,0.58,0.44,0.80,-5.61,-8.72,-2.84
0.18+,35.50,28.5,46.73,43.11,50.00,0.79,0.69,1.06,-3.83,-5.50,1.14


In [8]:

# ==== ATR帯×期間ばらつき（戦略別） ====
entry_exit = ensure_entry_exit()
exits = entry_exit.dropna(subset=['atr_entry','exit_net','strategy']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')

# 期間キー（優先: exit_timestamp -> timestamp_x -> timestamp）
ts_col = None
for cand in ('exit_timestamp', 'timestamp_x', 'timestamp'):
    if cand in exits.columns:
        ts_col = cand
        break
if ts_col is None:
    raise KeyError('時刻列が見つかりません (exit_timestamp/timestamp_x/timestamp)')

exits[ts_col] = pd.to_datetime(exits[ts_col])
exits['period'] = exits[ts_col].dt.to_period('M').dt.to_timestamp()
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)


def summarize_period(g):
    total = len(g)
    wins = g['exit_net'] > 0
    losses = g['exit_net'] < 0
    gross_profit = g.loc[wins, 'exit_net'].sum()
    gross_loss = g.loc[losses, 'exit_net'].sum()
    pf = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    expectancy = g['exit_net'].mean()
    return pd.Series({
        'trades': total,
        'win_rate_pct': win_rate * 100,
        'pf': pf,
        'expectancy': expectancy,
        'sum_net': g['exit_net'].sum(),
    })

period_stats_s = (
    exits.groupby(['atr_bucket','strategy','period'], observed=False)
    .apply(summarize_period)
    .reset_index()
    .sort_values(['atr_bucket','strategy','period'])
)

band_quantiles_s = period_stats_s.groupby(['atr_bucket','strategy'], observed=False).agg({
    'trades': ['mean', 'median'],
    'win_rate_pct': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'pf': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'expectancy': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
})
band_quantiles_s.columns = [
    'trades_mean', 'trades_median',
    'win_rate_median', 'win_rate_p25', 'win_rate_p75',
    'pf_median', 'pf_p25', 'pf_p75',
    'expectancy_median', 'expectancy_p25', 'expectancy_p75',
]
band_quantiles_s = band_quantiles_s.round(2)
print('--- period_stats_s sample ---')
print(period_stats_s.head())
band_quantiles_s



--- period_stats_s sample ---
  atr_bucket strategy     period  trades  win_rate_pct        pf  expectancy  sum_net
0  0.00-0.10      CCI 2025-01-01     9.0     88.888889  0.481878   -1.826667   -16.44
1  0.00-0.10      CCI 2025-02-01     6.0    100.000000       inf   23.038333   138.23
2  0.00-0.10      CCI 2025-03-01     5.0     60.000000  1.063890    0.856000     4.28
3  0.00-0.10      CCI 2025-04-01     3.0     66.666667  0.123059   -9.786667   -29.36
4  0.00-0.10      CCI 2025-05-01     7.0     28.571429  0.414926  -14.525714  -101.68


/tmp/ipykernel_4901/3537096212.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_period)
/home/anyo_/workspace/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/home/anyo_/workspace/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/home/anyo_/workspace/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4623: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t >= 0.5,
/home/anyo_/work

trades_mean  trades_median  win_rate_median  win_rate_p25  win_rate_p75  pf_median  pf_p25  \
atr_bucket strategy                                                                                               
0.00-0.10  CCI              8.12            6.5            79.90         65.00         84.72       1.61    0.47   
           MACD             7.75            7.0            55.04         50.00         66.67       0.49    0.08   
           MA_CROSS         2.12            2.0           100.00         62.50        100.00        inf    0.11   
           RSI              3.88            3.5            77.50         66.67        100.00       1.75    0.21   
           STOCH            6.75            6.5            73.21         65.91         77.68       1.35    0.85   
0.10-0.14  CCI             19.25           17.5            69.09         55.58         76.54       0.96    0.75   
           MACD            14.25           14.5            72.08         64.29         79.58       1.24    1.09   
           MA_CROSS         4.38            4.5            73.33         48.21         87.50       0.79    0.21   
           RSI              5.25            5.0            64.29         50.00         80.00       1.10    0.06   
           STOCH           10.62           11.0            61.82         56.08         65.91       0.73    0.44   
0.14-0.18  CCI             10.88           10.0            58.57         52.38         61.67       0.57    0.56   
           MACD            11.75           12.5            60.00         44.05         62.50       0.50    0.10   
           MA_CROSS         3.29            3.0            57.14         50.00         80.00       0.74    0.06   
           RSI              3.75            3.5            47.22         25.00         54.17       0.06    0.02   
           STOCH            6.88            6.5            58.57         46.88         64.18       0.26    0.08   
0.18+      CCI             15.12            9.0            43.25         40.09         45.83       0.39    0.07   
           MACD             8.88            7.0            50.00         42.71         50.83       1.00    0.82   
           MA_CROSS         3.57            3.0            60.00         34.72         83.33       1.59    0.37   
           RSI              4.25            3.0            50.00         46.88         62.50       1.28    0.98   
           STOCH            4.12            4.0            45.00         31.25         62.50       0.49    0.05   

                     pf_p75  expectancy_median  expectancy_p25  expectancy_p75  
atr_bucket strategy                                                             
0.00-0.10  CCI         2.46               3.91           -3.82            9.99  
           MACD        1.09              -9.71          -13.34           -0.44  
           MA_CROSS     NaN               1.95          -11.38           35.27  
           RSI          NaN               3.63           -6.27           13.26  
           STOCH       2.64               1.69           -1.06            7.66  
0.10-0.14  CCI         2.71              -0.51           -3.00           11.73  
           MACD        1.76               2.28            0.93            4.92  
           MA_CROSS     inf              -2.07           -9.29            2.07  
           RSI         3.16              -2.68          -15.88           12.78  
           STOCH       1.02              -4.03           -7.87            0.49  
0.14-0.18  CCI         1.56              -5.67           -6.88            6.26  
           MACD        1.02              -8.30          -11.14            0.35  
           MA_CROSS     NaN              -3.60          -16.36           12.30  
           RSI         0.30             -15.64          -25.42           -8.33  
           STOCH       1.33             -11.72          -14.03            2.90  
0.18+      CCI         0.56             -12.14          -14.22           -8.36  
           MACD        1.07     

In [9]:

# ==== ATR×Donchian帯域の損益集計 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width','strategy']).copy()
if don.empty:
    raise ValueError('ATR/Donchian/strategy が不足しています')
don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=DONCHIAN_BUCKET_BINS, labels=DONCHIAN_BUCKET_LABELS, right=False)
metrics_d = don.pivot_table(index=['atr_bucket','donchian_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['count', :]] = metrics_d.loc[:, idx['count', :]].astype(int)
if 'sum' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['sum', :]] = metrics_d.loc[:, idx['sum', :]].round(2)
if 'mean' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['mean', :]] = metrics_d.loc[:, idx['mean', :]].round(2)
metrics_d.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [10]:
# ==== ATR帯×戦略×EXIT理由の損益 ====
entry_exit = ensure_entry_exit()
if 'exit_reason' not in entry_exit.columns:
    entry_exit['exit_reason'] = 'UNKNOWN'
exits = entry_exit.dropna(subset=['atr_entry','strategy','exit_net']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')
exits['exit_reason'] = exits['exit_reason'].fillna('UNKNOWN')
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
exit_summary = (
    exits.groupby(['atr_bucket','strategy','exit_reason'], observed=False)
    .agg(count=('exit_net','size'), sum=('exit_net','sum'), mean=('exit_net','mean'))
    .reset_index()
    .sort_values(['atr_bucket','strategy','exit_reason'])
)
exit_summary[['sum','mean']] = exit_summary[['sum','mean']].round(2)
exit_summary['count'] = exit_summary['count'].astype(int)
exit_summary


,atr_bucket,strategy,exit_reason,count,sum,mean
0,0.00-0.10,CCI,STOP_BREAKEVEN,37,74.19,2.01
1,0.00-0.10,CCI,STOP_LOSS,16,-543.53,-33.97
2,0.00-0.10,CCI,TAKE_PROFIT,12,811.96,67.66
3,0.00-0.10,MACD,STOP_BREAKEVEN,29,58.64,2.02
4,0.00-0.10,MACD,STOP_LOSS,26,-879.00,-33.81
5,0.00-0.10,MACD,TAKE_PROFIT,7,478.89,68.41
6,0.00-0.10,MA_CROSS,STOP_BREAKEVEN,10,20.22,2.02
7,0.00-0.10,MA_CROSS,STOP_LOSS,4,-135.66,-33.92
8,0.00-0.10,MA_CROSS,TAKE_PROFIT,3,202.69,67.56
9,0.00-0.10,RSI,STOP_BREAKEVEN,18,36.62,2.03


In [11]:
# ==== ATR帯×戦略×EXIT理由（合計ピボット） ====
if 'exit_summary' not in globals():
    raise NameError('exit_summary が見つかりません (前セルを実行してください)')
exit_pivot = (
    exit_summary
    .pivot_table(index=['atr_bucket','strategy'], columns='exit_reason', values='sum', fill_value=0, observed=False)
    .sort_index()
    .round(2)
)
exit_pivot


exit_reason          STOP_BREAKEVEN  STOP_LOSS  TAKE_PROFIT
atr_bucket strategy                                        
0.00-0.10  CCI                74.19    -543.53       811.96
           MACD               58.64    -879.00       478.89
           MA_CROSS           20.22    -135.66       202.69
           RSI                36.62    -237.26       408.10
           STOCH              54.23    -540.80       740.35
0.10-0.14  CCI               143.60   -1751.41      2086.01
           MACD              118.82   -1084.59      1546.45
           MA_CROSS           36.36    -430.06       269.82
           RSI                42.52    -471.96       470.75
           STOCH              83.20   -1078.14       812.80
0.14-0.18  CCI                72.91   -1322.06       815.97
           MACD               83.09   -1425.48       740.98
           MA_CROSS           20.02    -300.95       270.01
           RSI                22.37    -576.53       138.00
           STOCH              44.57    -846.89       548.85
0.18+      CCI                58.70   -2367.66      1503.36
           MACD               26.24   -1223.50      1502.92
           MA_CROSS            7.72    -405.07       612.78
           RSI                10.19    -579.16       807.70
           STOCH              20.21    -646.99       270.10

In [12]:

# ==== Donchian×ATR×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width','strategy']).copy()
if don.empty:
    print('Donchian幅を持つレコードがありません')
else:
    don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
    don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=DONCHIAN_BUCKET_BINS, labels=DONCHIAN_BUCKET_LABELS, right=False)
    table_d = don.pivot_table(index=['atr_bucket','donchian_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
    idx = pd.IndexSlice
    if 'count' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['count', :]] = table_d.loc[:, idx['count', :]].astype(int)
    if 'sum' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['sum', :]] = table_d.loc[:, idx['sum', :]].round(2)
    if 'mean' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['mean', :]] = table_d.loc[:, idx['mean', :]].round(2)

    style = table_d.style
    value_levels = [lvl for lvl in ('sum','mean') if lvl in table_d.columns.get_level_values(0)]
    if value_levels:
        style = style.format('{:.2f}', subset=idx[:, idx[value_levels, :]])
    if 'count' in table_d.columns.get_level_values(0):
        style = style.format('{:d}', subset=idx[:, idx[['count'], :]])
    style.set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [13]:
# ==== 例: ATR帯域×戦略の損益集計 ====
exits = bt_df[bt_df['event'] == 'EXIT'].copy()
exits['atr_entry'] = exits['atr_entry'].astype(float)
bins = ATR_BUCKET_BINS
labels = ATR_BUCKET_LABELS
exits['atr_band'] = pd.cut(exits['atr_entry'], bins=bins, labels=labels, right=False)
pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0, observed=False)
pivot = pivot.round(2)
pivot.style.format('{:.2f}').set_table_attributes('style="table-layout: fixed; width: 100%;"')


strategy,CCI,MACD,MA_CROSS,RSI,STOCH
atr_band,,,,,
0.00-0.10,342.62,-341.47,87.25,207.46,253.78
0.10-0.14,478.20,580.68,-123.88,41.31,-182.14
0.14-0.18,-433.18,-601.41,-10.92,-416.16,-253.47
0.18+,-805.60,305.66,215.43,238.73,-356.68


In [14]:
# ==== BT全体指標のサマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')
bt_exits = bt_exits.sort_values('timestamp')

bt_exits['net'] = bt_exits['net'].astype(float)
bt_exits['pips'] = bt_exits['pips'].astype(float)

total_trades = len(bt_exits)
wins = bt_exits['net'] > 0
losses = bt_exits['net'] < 0

gross_profit = bt_exits.loc[wins, 'net'].sum()
gross_loss = bt_exits.loc[losses, 'net'].sum()
profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
win_rate = wins.sum() / total_trades if total_trades else float('nan')

avg_win = bt_exits.loc[wins, 'net'].mean() if wins.any() else 0.0
avg_loss = bt_exits.loc[losses, 'net'].mean() if losses.any() else 0.0
expectancy = bt_exits['net'].mean()

# 最大ドローダウンの算出
equity_curve = bt_exits['net'].cumsum()
equity_curve = pd.concat([pd.Series([0.0]), equity_curve], ignore_index=True)
rolling_peak = equity_curve.cummax()
drawdown = equity_curve - rolling_peak
max_drawdown = drawdown.min()
drawdown_pct = drawdown / rolling_peak.replace(0, pd.NA)
max_drawdown_pct = drawdown_pct.min()

summary = {
    'trades': total_trades,
    'wins': int(wins.sum()),
    'losses': int(losses.sum()),
    'win_rate_pct': win_rate * 100,
    'gross_profit': gross_profit,
    'gross_loss': gross_loss,
    'profit_factor': profit_factor,
    'avg_win': avg_win,
    'avg_loss': avg_loss,
    'expectancy': expectancy,
    'max_drawdown': max_drawdown,
    'max_drawdown_pct': max_drawdown_pct * 100,
}

label_map = {
    'trades': '総トレード数',
    'wins': '勝ち回数',
    'losses': '負け回数',
    'win_rate_pct': '勝率',
    'gross_profit': '総利益',
    'gross_loss': '総損失',
    'profit_factor': 'PF',
    'avg_win': '平均利益',
    'avg_loss': '平均損失',
    'expectancy': '期待値',
    'max_drawdown': '最大DD',
    'max_drawdown_pct': '最大DD(%)',
}
percent_keys = {'win_rate_pct', 'max_drawdown_pct'}
value_keys = {'profit_factor', 'avg_win', 'avg_loss', 'expectancy', 'gross_profit', 'gross_loss', 'max_drawdown'}

rows = []
for key in label_map:
    value = summary.get(key)
    if pd.isna(value):
        display_value = 'NaN'
    elif key in percent_keys:
        display_value = f"{value:.2f}%"
    elif key in value_keys:
        display_value = f"{value:.2f}"
    else:
        display_value = f"{int(value)}"
    rows.append({'指標': label_map[key], '値': display_value})

summary_df = pd.DataFrame(rows)
summary_df


,指標,値
0,総トレード数,1232
1,勝ち回数,734
2,負け回数,498
3,勝率,59.58%
4,総利益,16072.91
5,総損失,-16846.70
6,PF,0.95
7,平均利益,21.90
8,平均損失,-33.83
9,期待値,-0.63


In [15]:
# ==== 戦略別BT指標サマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')
if 'strategy' not in bt_exits.columns:
    raise KeyError('strategy 列が存在しません')

bt_exits['net'] = bt_exits['net'].astype(float)

def summarize_group(group):
    total = len(group)
    wins = group['net'] > 0
    losses = group['net'] < 0
    gross_profit = group.loc[wins, 'net'].sum()
    gross_loss = group.loc[losses, 'net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'net'].mean() if losses.any() else 0.0
    expectancy = group['net'].mean()
    return pd.Series({
        'trades': total,
        'wins': int(wins.sum()),
        'losses': int(losses.sum()),
        'win_rate_pct': win_rate * 100,
        'profit_factor': profit_factor,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'expectancy': expectancy,
    })

strategy_summary = (
    bt_exits.groupby('strategy', group_keys=False)
    .apply(summarize_group)
    .reset_index()
)

rename_map = {
    'strategy': 'Strategy',
    'trades': 'Trades',
    'wins': 'Wins',
    'losses': 'Losses',
    'win_rate_pct': 'Win Rate (%)',
    'profit_factor': 'PF',
    'avg_win': 'Avg Win',
    'avg_loss': 'Avg Loss',
    'expectancy': 'Expectancy',
}
strategy_summary = strategy_summary.rename(columns=rename_map)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}
strategy_summary.style.format(format_dict).hide(axis="index")


/tmp/ipykernel_4901/1671446051.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_group)


Strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
CCI,427.000000,250.000000,177.000000,58.55,0.93,22.27,-33.81,-0.98
MACD,341.000000,205.000000,136.000000,60.12,0.99,22.22,-33.92,-0.17
MA_CROSS,100.000000,62.000000,38.000000,62.00,1.13,23.22,-33.47,1.68
RSI,137.000000,82.000000,55.000000,59.85,1.04,23.61,-33.91,0.52
STOCH,227.000000,135.000000,92.000000,59.47,0.83,19.07,-33.84,-2.37


## TODO
- 週次/日次での State×BT 結合関数を関数化する
- 追加の特徴量（ATR増減、セッションフラグなど）を派生列として定義する
- ML モデル用に `merged` から特徴量テーブルを作成する

In [16]:
# ==== ノート別名保存（BTファイル名付き） ====
import nbformat
if 'bt_path' not in globals():
    raise NameError('bt_path が未定義です (データ読み込みセルを先に実行してください)')
base_name = bt_path.stem.replace(' ', '_')
dest_path = OUTPUT_NOTEBOOK_DIR / f"state_bt_analysis_{base_name}.ipynb"
with CURRENT_NOTEBOOK_PATH.open('r', encoding='utf-8') as fh:
    nb_data = nbformat.read(fh, as_version=4)
nbformat.write(nb_data, str(dest_path))
print(f'Saved notebook copy to: {dest_path}')


Saved notebook copy to: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/notebooks/state_bt_analysis_TradeLog_USDJPY_M15_ConfigOff_BE2_TrailOff_TP100_20250101_20250831.ipynb
